# Example for 2d Watershed Distance Trafo

Example for the watershed distance transform applied to a 3d probability map for each slice.
You can obtain the example probability map from:
https://www.dropbox.com/s/lddsbulvi82x902/neurocut_examples.tar.gz?dl=0

Note that this is anisotropic data, so this procedure is mote suited
than a 3d watershed distance transform (see example3d).
Feel free, to try this on your own data. 
However you may need to tweak some parameters
to get the best results.

In [1]:
import numpy as np
import vigra

from wsDtSegmentation import wsDtSegmentation

In [11]:
# 2d distance transform superpixel for the probability maps
pmap_path = "/path/to/neurocut_examples/probability_map.h5"
pmap_key  = "data"
pmaps = vigra.readHDF5(pmap_path, pmap_key)

# parameters for the watershed on distance trafo

# threshold for computing the distance trafo
threshold = 0.5
# minimal size of connected components that are taken into account
# for the distance trafo
min_mem = 50
# minimal size of segments in the result
min_seg = 75 
# sigma for smoothing the seed map
sig_seeds = 1.6
# sigma for smoothing the weight map
sig_weights = 2.0

segmentation = np.zeros_like(pmaps, dtype = np.uint32)
# we need an offset for each slice, because we need distinct ids in each slice
offset = 0
# iterate over the z-slices and perform the wsdt in each
for z in xrange(segmentation.shape[2]):
    segmentation[:,:,z] = wsDtSegmentation(
        pmaps[:,:,z], threshold,
        min_mem, min_seg,
        sig_seeds, sig_weights)
    # add the offset
    segmentation[:,:,z] += offset
    # get the new offset
    offset = np.max(segmentation)

# save the result
save_path = "/path/to/neurocut_examples/oversegmentation.h5"
save_key  = "superpixel"

vigra.writeHDF5(segmentation, save_path, save_key)